## Lyzr AI Therapist

This notebook provides a detailed guide and fully documented code examples to build a AI therapist using the Lyzr Agent API.

**1.  Installation of Lyzr Agent API**

First install the lyzr-agent-api with the help of pip command.


In [32]:
!pip install lyzr-agent-api

**2.  Setup and Initialization**

Import the necessary library and initialize your Lyzr API client.

In [33]:
from lyzr_agent_api import AgentAPI, AgentConfig
from lyzr_agent_api import EnvironmentConfig, FeatureConfig
from lyzr_agent_api import ChatRequest, OpenAPISchema

In [42]:
# X_API_KEY variable holds the Lyzr api key to authenticate the api requests, basically its a header token.

X_API_Key = "X_API_KEY"

In [35]:
# OPENAI_API_KEY variable have our openai api key

OPENAI_API_KEY = "OPENAI_API_KEY"

In [44]:
# We need to initialize the client for AgentAPI.
client = AgentAPI(
    base_url="https://agent.api.lyzr.app",
    x_api_key=X_API_Key,
    timeout=3600
)

print("Client has been initialized successfully")

Client has been initialized successfully


**3. Environment Creation**

Set up the agent's environment. 
The `create_env` function creates a new environment with specified configurations such as features as a module it uses `LONG_TERM_MEMORY`, tools such as `perplexity_search`, and a large language model (LLM) as `gpt-4o-mini`. It takes an environment name and optional API key, makes an API call to set up the environment, and returns the newly created environment's ID.

In [45]:
# This function will create the environment based on the given configuration
def create_env(env_name, openai_api_key=OPENAI_API_KEY):
    env_config = EnvironmentConfig(
        name=env_name,
        features=[
            FeatureConfig(
                type="LONG_TERM_MEMORY",
                config={"max_tries": 3},
                priority=0,
            )
        ],
        tools=["perplexity_search"],
        llm_config={"provider": "openai",
                    "model": "gpt-4o-mini",
                    "config": {
                        "temperature": 0.5,
                        "top_p": 0.9
                    },
                    "env": {
                        "OPENAI_API_KEY": openai_api_key
                    }},
    )

    response = client.create_environment_endpoint(json_body=env_config)
    if response:
      print('Environment has been created successfully')
      return response['env_id']

print("Environment is configured with LONG_TERM_MEMORY Module, tool as perplexity_search and LLM Model as gpt-4o-mini from OpenAI.")

Environment is configured with LONG_TERM_MEMORY Module, tool as perplexity_search and LLM Model as gpt-4o-mini from OpenAI.


**4. Agent Creation**

Define the configuration for your Content Creation Agent, outlining its purpose and capabilities in the system prompt.

The `create_ai_therapist_agent` function creates an agent in a specified environment by taking the environment ID. It configures the agent with a system prompt for AI Therapist and an agent description, then makes an API call to create the agent, returning the newly created agent's ID.


In [46]:
# This function will help us create agent based on specific environment
def create_ai_therapist_agent(env_id):
    agent_config = AgentConfig(
        env_id=env_id,
        system_prompt="You are an empathetic and supportive AI therapist. You help users manage their emotions, offer advice, and provide mental health support through thoughtful conversation.",
        name="AI Therapist Agent",
        agent_description="Offers mental health support, listens to users, and provides therapeutic advice."
    )
    agent_response = client.create_agent_endpoint(json_body=agent_config)
    return agent_response['agent_id']


print("Agent will need an environment id to configured successfully.")

Agent will need an environment id to configured successfully.


**5. User Interaction and Content Generation**

Create a function to handle user requests and provide reponses according to their concern.

---



In [47]:
def provide_therapy(user_id, agent_id, user_message):
    """Engages in a therapeutic conversation based on the user's message.

    Args:
        user_id (str): The user's ID.
        agent_id (str): The AI Therapist agent's ID.
        user_message (str): The user's message seeking therapy or advice.

    Returns:
        str: The AI Therapist's response.
    """

    json_body = ChatRequest(
        user_id=user_id,  # Your user_id, you will get it from the studio.
        agent_id=agent_id,
        message=user_message,
    )

    chat = client.chat_with_agent(json_body=json_body)
    return chat['response']


In [53]:
# Example Usage
user_id = 'user_id'  # Replace with your actual user ID
env_id = create_env(env_name='AI Therapist Environment')
agent_id = create_ai_therapist_agent(env_id=env_id)
user_message = "I'm feeling stressed and anxious about my work. How can I manage these feelings?"

# Engaging in a therapeutic conversation
therapist_response = provide_therapy(user_id=user_id,
                                     agent_id=agent_id,
                                     user_message=user_message)


Key Points and Further Enhancements for **AI Therapist**:

- **Prompt Engineering**: Tailor prompts to elicit a specific therapeutic approach or response. For example, you could prompt the AI to offer "mindfulness exercises" or "cognitive behavioral therapy techniques" depending on the user’s needs.
  
- **Therapy Techniques**: Expand the agent's knowledge to cover a wider range of therapy techniques such as mindfulness, stress management, emotional regulation, or even offer supportive listening for mental well-being.

- **Iterative Therapy Sessions**: Allow users to engage in continuous conversations, where they can provide feedback or updates, and the agent adapts by providing deeper insights or further therapeutic exercises.

- **Personalization**: The agent could be enhanced to recognize patterns over time and personalize its responses based on previous interactions, providing a tailored therapeutic experience.

- **Resource Suggestions**: Implement the capability to suggest helpful resources like articles, exercises, or helplines for more serious issues beyond what the agent can handle.

In [54]:
print(therapist_response)

It's completely understandable to feel stressed and anxious about work. Many people experience these feelings, especially when under pressure. It's important to acknowledge your emotions rather than push them aside.

Can you share more about what specifically is causing you stress at work? Is it deadlines, workload, interpersonal relationships, or something else? This might help us explore some tailored strategies together. 

In the meantime, here are a few general techniques that might help:

1. **Deep Breathing**: Take a moment to practice deep breathing exercises. Inhale deeply through your nose for a count of four, hold it for four, and then exhale slowly through your mouth for a count of four. This can help calm your nervous system.

2. **Time Management**: Break your tasks into smaller, manageable parts and prioritize them. Sometimes, just knowing you have a plan can ease anxiety.

3. **Mindfulness & Meditation**: Consider practicing mindfulness or short meditation sessions. This